In [1]:
import pandas as pd
import numpy as np
import os
import imageio

from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Maximum
from keras.layers import ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize as imresize
from tqdm import tqdm


Using TensorFlow backend.


In [2]:
CLASS = {
    'Black-grass': 0,
    'Charlock': 1,
    'Cleavers': 2,
    'Common Chickweed': 3,
    'Common wheat': 4,
    'Fat Hen': 5,
    'Loose Silky-bent': 6,
    'Maize': 7,
    'Scentless Mayweed': 8,
    'Shepherds Purse': 9,
    'Small-flowered Cranesbill': 10,
    'Sugar beet': 11
}

INV_CLASS = {
    0: 'Black-grass',
    1: 'Charlock',
    2: 'Cleavers',
    3: 'Common Chickweed',
    4: 'Common wheat',
    5: 'Fat Hen',
    6: 'Loose Silky-bent',
    7: 'Maize',
    8: 'Scentless Mayweed',
    9: 'Shepherds Purse',
    10: 'Small-flowered Cranesbill',
    11: 'Sugar beet'
}

In [3]:
BATCH_SIZE = 512
EPOCHS = 30
RANDOM_STATE = 11


In [4]:
def img_reshape(img):
    img = imresize(img, (51, 51, 3))
    return img



In [5]:
# get image tag
def img_label(path):
    print("Image Label below")
    print(str(str(path.split('\\')[-1])))
    return str(str(path.split('\\')[-1]))



In [6]:
# get plant class on image
def img_class(path):
    print('Image class is ',str(path.split('/')[-2]))
    return str(path.split('\\')[-2])

In [7]:
def fill_dict(paths, some_dict):
    text = ''
    if 'train' in paths[0]:
        text = 'Start fill train_dict'
    elif 'test' in paths[0]:
        text = 'Start fill test_dict'

    for p in tqdm(paths, ascii=True, ncols=85, desc=text):
        img = imageio.imread(p)
        img = img_reshape(img)
        some_dict['image'].append(img)
        some_dict['label'].append(img_label(p))
        if 'train' in paths[0]:
            print('Image ',img_class(p))
            some_dict['class'].append(img_class(p))

    return some_dict

In [8]:
def reader():
    file_ext = []
    train_path = []
    test_path = []

    for root, dirs, files in os.walk('./input'):
        if dirs != []:
            print('Root:\n'+str(root))
            print('Dirs:\n'+str(dirs))
        else:
            for f in files:
                ext = os.path.splitext(str(f))[1][1:]
                #print('below is root')
                #print(root)
                
                if ext not in file_ext:
                    file_ext.append(ext)

                if 'train' in root:
                    #print('train presnet in root',root)
                    path = os.path.join(root, f)
                    train_path.append(path)
                elif 'test' in root:
                    path = os.path.join(root, f)
                    test_path.append(path)
    train_dict = {
        'image': [],
        'label': [],
        'class': []
    }
    test_dict = {
        'image': [],
        'label': []
    }

    train_dict = fill_dict(train_path, train_dict)
    test_dict = fill_dict(test_path, test_dict)
    return train_dict, test_dict

In [9]:
train_dict, test_dict = reader()

Root:
./input
Dirs:
['test', 'train']
Root:
./input\train
Dirs:
['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']


Start fill train_dict:   0%|                                | 0/4750 [00:00<?, ?it/s]

Image Label below
0050f38b3.png
Image class is  .
Image  Black-grass
Image class is  .
Image Label below
0183fdf68.png
Image class is  .
Image  Black-grass
Image class is  .
Image Label below
0260cffa8.png
Image class is  .
Image  Black-grass
Image class is  .


Start fill train_dict:   0%|                        | 3/4750 [00:00<05:00, 15.80it/s]

Image Label below
05eedce4d.png
Image class is  .
Image  Black-grass
Image class is  .
Image Label below
075d004bc.png
Image class is  .
Image  Black-grass
Image class is  .
Image Label below
078eae073.png
Image class is  .
Image  Black-grass
Image class is  .


...

Start fill test_dict:  97%|######################4| 774/794 [00:07<00:00, 103.53it/s]

Image Label below
fadc6adbc.png
Image Label below
fb022edf9.png
Image Label below
fba8fc78a.png
Image Label below
fbf88b6be.png
Image Label below
fc3e58836.png
Image Label below
fc6f686fb.png
Image Label below
fd253a74e.png
Image Label below
fd87b36ae.png
Image Label below
fd925f542.png
Image Label below
fda0b5c38.png
Image Label below
fda39e16f.png


Start fill test_dict:  99%|######################7| 785/794 [00:07<00:00, 103.60it/s]

Image Label below
fdea6b119.png
Image Label below
fe29629fb.png
Image Label below
fe9e87b78.png
Image Label below
fea1d13d6.png
Image Label below
fea355851.png
Image Label below
fea3da57c.png
Image Label below
fef2ade8c.png
Image Label below
ff65bc002.png
Image Label below
ffc6f8527.png


Start fill test_dict: 100%|#######################| 794/794 [00:07<00:00, 101.80it/s]


In [10]:
train_dict['class']

['Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-

In [17]:
X = np.array(train_dict['image'])
y = to_categorical(np.array([CLASS[l] for l in train_dict['class']]))

In [18]:
x_train, x_test, y_train, y_test = train_test_split(
                                                        X,
                                                        y,
                                                        shuffle=True,
                                                        train_size=0.8,
                                                        random_state=RANDOM_STATE
                                                        )

In [19]:
import keras

In [20]:

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D,BatchNormalization

In [21]:
 #Define Model
model3 = Sequential()
model3.add(Convolution2D(51, 3, 3, input_shape=(51, 51, 3)))
model3.add(Activation('relu'))
# 2nd Conv Layer
model3.add(Convolution2D(256, 3, 3))
model3.add(Activation('relu'))
    
model3.add(Convolution2D(256, 3, 3))
model3.add(Activation('relu'))

model3.add(MaxPooling2D(pool_size=(3,3)))

# Dropout
model3.add(Dropout(0.3))
model3.add(BatchNormalization())

model3.add(Convolution2D(256, 3, 3))
model3.add(Activation('relu'))
    
model3.add(Convolution2D(256, 3, 3))
model3.add(Activation('relu'))
    
    
# Dropout
model3.add(Dropout(0.3))
model3.add(BatchNormalization())
    
    # Max Pooling
model3.add(MaxPooling2D(pool_size=(3,3)))
    
    # Dropout
model3.add(Dropout(0.1))
model3.add(BatchNormalization())
    
    
model3.add(Convolution2D(256, 3, 3))
model3.add(Activation('relu'))

model3.add(Dropout(0.3))
model3.add(BatchNormalization())
    # Fully Connected Layer

C:\Users\Gauravski\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(51, (3, 3), input_shape=(51, 51, 3...)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Gauravski\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3))`
  
C:\Users\Gauravski\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3))`
  if __name__ == '__main__':
C:\Users\Gauravski\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3))`
C:\Users\Gauravski\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3))`
C:\Users\Gauravski\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update 

In [22]:
    model3.add(Flatten())
    model3.add(Dense(256))
    model3.add(Activation('relu'))

    model3.add(Dropout(0.1))
    model3.add(BatchNormalization())
    # Prediction Layer
    model3.add(Dense(12))
    model3.add(Activation('softmax'))

In [23]:
model3.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

W1103 16:51:44.516624  9560 deprecation_wrapper.py:119] From C:\Users\Gauravski\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [24]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=7, verbose=1, mode='auto')
callback_list = [early_stopping]

In [25]:
model3.fit(x_train, y_train, batch_size=BATCH_SIZE, nb_epoch=60, 
              validation_data=(x_test, y_test))

C:\Users\Gauravski\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  
W1103 16:51:46.539939  9560 deprecation.py:323] From C:\Users\Gauravski\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 3800 samples, validate on 950 samples
Epoch 1/60
3800/3800 [==============================] - ETA: 41s - loss: 3.2154 - acc: 0.08 - ETA: 18s - loss: 3.1711 - acc: 0.10 - ETA: 10s - loss: 3.1246 - acc: 0.09 - ETA: 6s - loss: 3.0348 - acc: 0.1167 - ETA: 3s - loss: 2.9539 - acc: 0.131 - ETA: 2s - loss: 2.8730 - acc: 0.146 - ETA: 0s - loss: 2.7846 - acc: 0.162 - 13s 3ms/step - loss: 2.7498 - acc: 0.1703 - val_loss: 6.5256 - val_acc: 0.0979
Epoch 2/60
3800/3800 [==============================] - ETA: 2s - loss: 2.4276 - acc: 0.230 - ETA: 2s - loss: 2.2373 - acc: 0.268 - ETA: 1s - loss: 2.1181 - acc: 0.300 - ETA: 1s - loss: 2.0101 - acc: 0.324 - ETA: 1s - loss: 1.9431 - acc: 0.343 - ETA: 0s - loss: 1.9005 - acc: 0.358 - ETA: 0s - loss: 1.8873 - acc: 0.368 - 3s 892us/step - loss: 1.8721 - acc: 0.3695 - val_loss: 2.8394 - val_acc: 0.2874
Epoch 3/60
3800/3800 [==============================] - ETA: 2s - loss: 1.4364 - acc: 0.502 - ETA: 2s - loss: 1.4709 - acc: 0.508 - ETA: 1s - loss: 1

3800/3800 [==============================] - ETA: 2s - loss: 0.3175 - acc: 0.859 - ETA: 2s - loss: 0.3362 - acc: 0.862 - ETA: 1s - loss: 0.3537 - acc: 0.860 - ETA: 1s - loss: 0.3331 - acc: 0.872 - ETA: 1s - loss: 0.3170 - acc: 0.881 - ETA: 0s - loss: 0.3046 - acc: 0.886 - ETA: 0s - loss: 0.2959 - acc: 0.890 - 4s 927us/step - loss: 0.2931 - acc: 0.8908 - val_loss: 0.8944 - val_acc: 0.7600
Epoch 22/60
3800/3800 [==============================] - ETA: 2s - loss: 0.2067 - acc: 0.929 - ETA: 2s - loss: 0.2048 - acc: 0.927 - ETA: 1s - loss: 0.1991 - acc: 0.931 - ETA: 1s - loss: 0.1977 - acc: 0.931 - ETA: 1s - loss: 0.2023 - acc: 0.929 - ETA: 0s - loss: 0.2020 - acc: 0.931 - ETA: 0s - loss: 0.1976 - acc: 0.933 - 3s 912us/step - loss: 0.1995 - acc: 0.9326 - val_loss: 1.7980 - val_acc: 0.5221
Epoch 23/60
3800/3800 [==============================] - ETA: 2s - loss: 0.2660 - acc: 0.896 - ETA: 2s - loss: 0.2578 - acc: 0.907 - ETA: 1s - loss: 0.2541 - acc: 0.909 - ETA: 1s - loss: 0.2758 - acc: 0.898

3800/3800 [==============================] - ETA: 2s - loss: 0.1087 - acc: 0.960 - ETA: 2s - loss: 0.0926 - acc: 0.962 - ETA: 2s - loss: 0.0863 - acc: 0.966 - ETA: 1s - loss: 0.0789 - acc: 0.971 - ETA: 1s - loss: 0.0730 - acc: 0.975 - ETA: 0s - loss: 0.0700 - acc: 0.977 - ETA: 0s - loss: 0.0667 - acc: 0.978 - 4s 945us/step - loss: 0.0656 - acc: 0.9795 - val_loss: 2.4209 - val_acc: 0.5126
Epoch 42/60
3800/3800 [==============================] - ETA: 2s - loss: 0.0696 - acc: 0.970 - ETA: 2s - loss: 0.0571 - acc: 0.978 - ETA: 1s - loss: 0.0582 - acc: 0.979 - ETA: 1s - loss: 0.0550 - acc: 0.982 - ETA: 1s - loss: 0.0575 - acc: 0.981 - ETA: 0s - loss: 0.0659 - acc: 0.977 - ETA: 0s - loss: 0.0699 - acc: 0.976 - 4s 923us/step - loss: 0.0747 - acc: 0.9747 - val_loss: 0.9436 - val_acc: 0.7432
Epoch 43/60
3800/3800 [==============================] - ETA: 2s - loss: 0.0955 - acc: 0.970 - ETA: 2s - loss: 0.0767 - acc: 0.974 - ETA: 1s - loss: 0.0617 - acc: 0.979 - ETA: 1s - loss: 0.0559 - acc: 0.982

In [27]:
gen = ImageDataGenerator(
            rotation_range=360.,
            width_shift_range=0.3,
            height_shift_range=0.3,
            zoom_range=0.3,
            horizontal_flip=True,
            vertical_flip=True
    )

In [28]:
model3.fit_generator(gen.flow(x_train, y_train,batch_size=640),
               steps_per_epoch=x_train.shape[0],
               epochs=4,
               verbose=1,
               shuffle=True,
               validation_data=(x_test, y_test),
               )

Epoch 1/4


 203/3800 [>.............................] - ETA: 5:11:55 - loss: 3.4543 - acc: 0.50 - ETA: 2:51:50 - loss: 3.1193 - acc: 0.50 - ETA: 2:05:07 - loss: 3.0157 - acc: 0.49 - ETA: 1:41:47 - loss: 2.9289 - acc: 0.49 - ETA: 1:27:43 - loss: 2.7621 - acc: 0.50 - ETA: 1:59:53 - loss: 2.6136 - acc: 0.51 - ETA: 1:47:17 - loss: 2.4690 - acc: 0.53 - ETA: 1:37:51 - loss: 2.3508 - acc: 0.54 - ETA: 1:30:29 - loss: 2.2624 - acc: 0.54 - ETA: 1:24:37 - loss: 2.1676 - acc: 0.55 - ETA: 1:19:51 - loss: 2.0972 - acc: 0.56 - ETA: 1:15:41 - loss: 2.0344 - acc: 0.57 - ETA: 1:12:17 - loss: 1.9756 - acc: 0.57 - ETA: 1:09:23 - loss: 1.9283 - acc: 0.58 - ETA: 1:06:52 - loss: 1.8671 - acc: 0.58 - ETA: 1:04:40 - loss: 1.8224 - acc: 0.59 - ETA: 1:02:43 - loss: 1.7853 - acc: 0.59 - ETA: 1:00:53 - loss: 1.7458 - acc: 0.60 - ETA: 59:20 - loss: 1.7142 - acc: 0.6026 - ETA: 57:57 - loss: 1.6747 - acc: 0.60 - ETA: 56:41 - loss: 1.6454 - acc: 0.61 - ETA: 55:33 - loss: 1.6168 - acc: 0.61 - ETA: 54:30 - loss: 1.5933 - acc: 0.61

 407/3800 [==>...........................] - ETA: 32:42 - loss: 0.7472 - acc: 0.76 - ETA: 32:41 - loss: 0.7456 - acc: 0.76 - ETA: 32:40 - loss: 0.7442 - acc: 0.76 - ETA: 32:39 - loss: 0.7427 - acc: 0.76 - ETA: 32:37 - loss: 0.7414 - acc: 0.76 - ETA: 32:36 - loss: 0.7396 - acc: 0.76 - ETA: 32:35 - loss: 0.7384 - acc: 0.76 - ETA: 32:33 - loss: 0.7377 - acc: 0.76 - ETA: 32:32 - loss: 0.7371 - acc: 0.76 - ETA: 32:31 - loss: 0.7359 - acc: 0.76 - ETA: 32:30 - loss: 0.7347 - acc: 0.76 - ETA: 32:29 - loss: 0.7333 - acc: 0.76 - ETA: 32:27 - loss: 0.7321 - acc: 0.76 - ETA: 32:26 - loss: 0.7308 - acc: 0.76 - ETA: 32:25 - loss: 0.7295 - acc: 0.76 - ETA: 32:24 - loss: 0.7281 - acc: 0.76 - ETA: 32:23 - loss: 0.7270 - acc: 0.76 - ETA: 32:22 - loss: 0.7257 - acc: 0.76 - ETA: 32:21 - loss: 0.7247 - acc: 0.76 - ETA: 32:19 - loss: 0.7230 - acc: 0.76 - ETA: 32:18 - loss: 0.7217 - acc: 0.76 - ETA: 32:17 - loss: 0.7205 - acc: 0.76 - ETA: 32:16 - loss: 0.7193 - acc: 0.77 - ETA: 32:15 - loss: 0.7183 - acc: 0.

 611/3800 [===>..........................] - ETA: 29:44 - loss: 0.5675 - acc: 0.81 - ETA: 29:43 - loss: 0.5669 - acc: 0.81 - ETA: 29:42 - loss: 0.5663 - acc: 0.81 - ETA: 29:42 - loss: 0.5657 - acc: 0.81 - ETA: 29:41 - loss: 0.5653 - acc: 0.81 - ETA: 29:40 - loss: 0.5646 - acc: 0.81 - ETA: 29:40 - loss: 0.5641 - acc: 0.81 - ETA: 29:39 - loss: 0.5635 - acc: 0.81 - ETA: 29:38 - loss: 0.5629 - acc: 0.81 - ETA: 29:38 - loss: 0.5626 - acc: 0.81 - ETA: 29:37 - loss: 0.5622 - acc: 0.81 - ETA: 29:36 - loss: 0.5615 - acc: 0.81 - ETA: 29:36 - loss: 0.5609 - acc: 0.81 - ETA: 29:35 - loss: 0.5603 - acc: 0.81 - ETA: 29:34 - loss: 0.5596 - acc: 0.81 - ETA: 29:34 - loss: 0.5590 - acc: 0.81 - ETA: 29:33 - loss: 0.5583 - acc: 0.81 - ETA: 29:32 - loss: 0.5577 - acc: 0.81 - ETA: 29:31 - loss: 0.5571 - acc: 0.81 - ETA: 29:31 - loss: 0.5566 - acc: 0.81 - ETA: 29:30 - loss: 0.5560 - acc: 0.81 - ETA: 29:30 - loss: 0.5553 - acc: 0.81 - ETA: 29:29 - loss: 0.5547 - acc: 0.81 - ETA: 29:28 - loss: 0.5542 - acc: 0.

 815/3800 [=====>........................] - ETA: 27:36 - loss: 0.4751 - acc: 0.84 - ETA: 27:35 - loss: 0.4748 - acc: 0.84 - ETA: 27:35 - loss: 0.4745 - acc: 0.84 - ETA: 27:34 - loss: 0.4741 - acc: 0.84 - ETA: 27:34 - loss: 0.4738 - acc: 0.84 - ETA: 27:33 - loss: 0.4735 - acc: 0.84 - ETA: 27:32 - loss: 0.4731 - acc: 0.84 - ETA: 27:32 - loss: 0.4727 - acc: 0.84 - ETA: 27:31 - loss: 0.4723 - acc: 0.84 - ETA: 27:31 - loss: 0.4720 - acc: 0.84 - ETA: 27:30 - loss: 0.4716 - acc: 0.84 - ETA: 27:30 - loss: 0.4713 - acc: 0.84 - ETA: 27:29 - loss: 0.4708 - acc: 0.84 - ETA: 27:28 - loss: 0.4704 - acc: 0.84 - ETA: 27:28 - loss: 0.4700 - acc: 0.84 - ETA: 27:27 - loss: 0.4697 - acc: 0.84 - ETA: 27:27 - loss: 0.4693 - acc: 0.84 - ETA: 27:26 - loss: 0.4689 - acc: 0.84 - ETA: 27:25 - loss: 0.4686 - acc: 0.84 - ETA: 27:25 - loss: 0.4681 - acc: 0.84 - ETA: 27:24 - loss: 0.4677 - acc: 0.84 - ETA: 27:24 - loss: 0.4674 - acc: 0.84 - ETA: 27:23 - loss: 0.4670 - acc: 0.84 - ETA: 27:23 - loss: 0.4665 - acc: 0.

...

3060/3800 [=======================>......] - ETA: 8:09 - loss: 0.0320 - acc: 0.989 - ETA: 8:09 - loss: 0.0320 - acc: 0.989 - ETA: 8:08 - loss: 0.0320 - acc: 0.989 - ETA: 8:08 - loss: 0.0320 - acc: 0.989 - ETA: 8:07 - loss: 0.0320 - acc: 0.989 - ETA: 8:07 - loss: 0.0320 - acc: 0.989 - ETA: 8:06 - loss: 0.0320 - acc: 0.989 - ETA: 8:06 - loss: 0.0320 - acc: 0.989 - ETA: 8:05 - loss: 0.0320 - acc: 0.989 - ETA: 8:05 - loss: 0.0320 - acc: 0.989 - ETA: 8:04 - loss: 0.0320 - acc: 0.989 - ETA: 8:04 - loss: 0.0320 - acc: 0.989 - ETA: 8:03 - loss: 0.0320 - acc: 0.989 - ETA: 8:03 - loss: 0.0320 - acc: 0.989 - ETA: 8:02 - loss: 0.0320 - acc: 0.989 - ETA: 8:02 - loss: 0.0320 - acc: 0.989 - ETA: 8:01 - loss: 0.0320 - acc: 0.989 - ETA: 8:01 - loss: 0.0320 - acc: 0.989 - ETA: 8:00 - loss: 0.0320 - acc: 0.989 - ETA: 7:59 - loss: 0.0320 - acc: 0.989 - ETA: 7:59 - loss: 0.0320 - acc: 0.989 - ETA: 7:58 - loss: 0.0320 - acc: 0.989 - ETA: 7:58 - loss: 0.0320 - acc: 0.989 - ETA: 7:57 - loss: 0.0320 - acc: 0.9

3264/3800 [========================>.....] - ETA: 6:23 - loss: 0.0318 - acc: 0.989 - ETA: 6:22 - loss: 0.0318 - acc: 0.989 - ETA: 6:22 - loss: 0.0318 - acc: 0.989 - ETA: 6:21 - loss: 0.0318 - acc: 0.989 - ETA: 6:21 - loss: 0.0318 - acc: 0.989 - ETA: 6:20 - loss: 0.0318 - acc: 0.989 - ETA: 6:20 - loss: 0.0318 - acc: 0.989 - ETA: 6:19 - loss: 0.0318 - acc: 0.989 - ETA: 6:19 - loss: 0.0318 - acc: 0.989 - ETA: 6:18 - loss: 0.0318 - acc: 0.989 - ETA: 6:18 - loss: 0.0318 - acc: 0.989 - ETA: 6:17 - loss: 0.0318 - acc: 0.989 - ETA: 6:17 - loss: 0.0318 - acc: 0.989 - ETA: 6:16 - loss: 0.0318 - acc: 0.989 - ETA: 6:16 - loss: 0.0318 - acc: 0.989 - ETA: 6:15 - loss: 0.0318 - acc: 0.989 - ETA: 6:15 - loss: 0.0318 - acc: 0.989 - ETA: 6:14 - loss: 0.0318 - acc: 0.989 - ETA: 6:13 - loss: 0.0318 - acc: 0.989 - ETA: 6:13 - loss: 0.0318 - acc: 0.989 - ETA: 6:12 - loss: 0.0318 - acc: 0.989 - ETA: 6:12 - loss: 0.0318 - acc: 0.989 - ETA: 6:11 - loss: 0.0318 - acc: 0.989 - ETA: 6:11 - loss: 0.0318 - acc: 0.9

3468/3800 [==========================>...] - ETA: 4:37 - loss: 0.0318 - acc: 0.989 - ETA: 4:36 - loss: 0.0318 - acc: 0.989 - ETA: 4:36 - loss: 0.0318 - acc: 0.989 - ETA: 4:35 - loss: 0.0318 - acc: 0.989 - ETA: 4:35 - loss: 0.0318 - acc: 0.989 - ETA: 4:34 - loss: 0.0318 - acc: 0.989 - ETA: 4:34 - loss: 0.0318 - acc: 0.989 - ETA: 4:33 - loss: 0.0318 - acc: 0.989 - ETA: 4:33 - loss: 0.0318 - acc: 0.989 - ETA: 4:32 - loss: 0.0318 - acc: 0.989 - ETA: 4:31 - loss: 0.0318 - acc: 0.989 - ETA: 4:31 - loss: 0.0318 - acc: 0.989 - ETA: 4:30 - loss: 0.0318 - acc: 0.989 - ETA: 4:30 - loss: 0.0318 - acc: 0.989 - ETA: 4:29 - loss: 0.0318 - acc: 0.989 - ETA: 4:29 - loss: 0.0318 - acc: 0.989 - ETA: 4:28 - loss: 0.0318 - acc: 0.989 - ETA: 4:28 - loss: 0.0318 - acc: 0.989 - ETA: 4:27 - loss: 0.0318 - acc: 0.989 - ETA: 4:27 - loss: 0.0318 - acc: 0.989 - ETA: 4:26 - loss: 0.0317 - acc: 0.989 - ETA: 4:26 - loss: 0.0318 - acc: 0.989 - ETA: 4:25 - loss: 0.0317 - acc: 0.989 - ETA: 4:25 - loss: 0.0317 - acc: 0.9

3672/3800 [===========================>..] - ETA: 2:51 - loss: 0.0316 - acc: 0.989 - ETA: 2:50 - loss: 0.0316 - acc: 0.989 - ETA: 2:50 - loss: 0.0316 - acc: 0.989 - ETA: 2:49 - loss: 0.0316 - acc: 0.989 - ETA: 2:49 - loss: 0.0316 - acc: 0.989 - ETA: 2:48 - loss: 0.0316 - acc: 0.989 - ETA: 2:48 - loss: 0.0316 - acc: 0.989 - ETA: 2:47 - loss: 0.0316 - acc: 0.989 - ETA: 2:47 - loss: 0.0316 - acc: 0.989 - ETA: 2:46 - loss: 0.0316 - acc: 0.989 - ETA: 2:46 - loss: 0.0316 - acc: 0.989 - ETA: 2:45 - loss: 0.0316 - acc: 0.989 - ETA: 2:45 - loss: 0.0316 - acc: 0.989 - ETA: 2:44 - loss: 0.0316 - acc: 0.989 - ETA: 2:44 - loss: 0.0316 - acc: 0.989 - ETA: 2:43 - loss: 0.0316 - acc: 0.989 - ETA: 2:43 - loss: 0.0316 - acc: 0.989 - ETA: 2:42 - loss: 0.0316 - acc: 0.989 - ETA: 2:41 - loss: 0.0316 - acc: 0.989 - ETA: 2:41 - loss: 0.0316 - acc: 0.989 - ETA: 2:40 - loss: 0.0316 - acc: 0.989 - ETA: 2:40 - loss: 0.0316 - acc: 0.989 - ETA: 2:39 - loss: 0.0316 - acc: 0.989 - ETA: 2:39 - loss: 0.0316 - acc: 0.9

3800/3800 [==============================] - ETA: 1:05 - loss: 0.0315 - acc: 0.989 - ETA: 1:05 - loss: 0.0315 - acc: 0.989 - ETA: 1:04 - loss: 0.0315 - acc: 0.989 - ETA: 1:04 - loss: 0.0315 - acc: 0.989 - ETA: 1:03 - loss: 0.0315 - acc: 0.989 - ETA: 1:03 - loss: 0.0315 - acc: 0.989 - ETA: 1:02 - loss: 0.0315 - acc: 0.989 - ETA: 1:02 - loss: 0.0315 - acc: 0.989 - ETA: 1:01 - loss: 0.0315 - acc: 0.989 - ETA: 1:01 - loss: 0.0315 - acc: 0.989 - ETA: 1:00 - loss: 0.0315 - acc: 0.989 - ETA: 59s - loss: 0.0315 - acc: 0.989 - ETA: 59s - loss: 0.0315 - acc: 0.98 - ETA: 58s - loss: 0.0315 - acc: 0.98 - ETA: 58s - loss: 0.0315 - acc: 0.98 - ETA: 57s - loss: 0.0315 - acc: 0.98 - ETA: 57s - loss: 0.0315 - acc: 0.98 - ETA: 56s - loss: 0.0315 - acc: 0.98 - ETA: 56s - loss: 0.0315 - acc: 0.98 - ETA: 55s - loss: 0.0315 - acc: 0.98 - ETA: 55s - loss: 0.0315 - acc: 0.98 - ETA: 54s - loss: 0.0315 - acc: 0.98 - ETA: 54s - loss: 0.0315 - acc: 0.98 - ETA: 53s - loss: 0.0315 - acc: 0.98 - ETA: 53s - loss: 0.0

In [29]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model3.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

In [32]:
model3.evaluate(x_train,y_train)


3800/3800 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 317us/step


[0.05097707171025257, 0.9847368421052631]

In [31]:
model3.evaluate(x_test,y_test)


950/950 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 639us/step


[0.31404918504388707, 0.9378947374695226]

In [30]:

for layer in model3.layers:
  print (colored(layer.name, 'blue'),colored(layer.trainable, 'red'))

conv2d_7 False
activation_9 False
conv2d_8 False
activation_10 False
conv2d_9 False
activation_11 False
max_pooling2d_3 False
dropout_6 False
batch_normalization_6 False
conv2d_10 False
activation_12 False
conv2d_11 False
activation_13 False
dropout_7 False
batch_normalization_7 False
max_pooling2d_4 False
dropout_8 False
batch_normalization_8 False
conv2d_12 False
activation_14 False
dropout_9 False
batch_normalization_9 False
flatten_2 False
dense_3 True
activation_15 False
dropout_10 False
batch_normalization_10 False
dense_4 True
activation_16 False


In [33]:

model3.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [35]:
model3.fit_generator(gen.flow(x_train, y_train,batch_size=640),
               steps_per_epoch=x_train.shape[0],
               epochs=1,
               verbose=1,
               shuffle=True,
               validation_data=(x_test, y_test),
               )

Epoch 1/1


 204/3800 [>.............................] - ETA: 32:48 - loss: 0.0245 - acc: 0.99 - ETA: 26:27 - loss: 0.0218 - acc: 0.99 - ETA: 24:55 - loss: 0.0202 - acc: 0.99 - ETA: 24:06 - loss: 0.0221 - acc: 0.99 - ETA: 23:38 - loss: 0.0205 - acc: 0.99 - ETA: 23:00 - loss: 0.0199 - acc: 0.99 - ETA: 22:53 - loss: 0.0189 - acc: 0.99 - ETA: 22:45 - loss: 0.0187 - acc: 0.99 - ETA: 22:37 - loss: 0.0202 - acc: 0.99 - ETA: 22:31 - loss: 0.0214 - acc: 0.99 - ETA: 22:27 - loss: 0.0215 - acc: 0.99 - ETA: 22:13 - loss: 0.0226 - acc: 0.99 - ETA: 22:13 - loss: 0.0230 - acc: 0.99 - ETA: 22:10 - loss: 0.0238 - acc: 0.99 - ETA: 22:08 - loss: 0.0235 - acc: 0.99 - ETA: 22:05 - loss: 0.0235 - acc: 0.99 - ETA: 22:04 - loss: 0.0231 - acc: 0.99 - ETA: 21:56 - loss: 0.0242 - acc: 0.99 - ETA: 21:57 - loss: 0.0238 - acc: 0.99 - ETA: 21:56 - loss: 0.0235 - acc: 0.99 - ETA: 21:55 - loss: 0.0236 - acc: 0.99 - ETA: 21:54 - loss: 0.0232 - acc: 0.99 - ETA: 21:53 - loss: 0.0228 - acc: 0.99 - ETA: 21:47 - loss: 0.0230 - acc: 0.

 408/3800 [==>...........................] - ETA: 20:31 - loss: 0.0231 - acc: 0.99 - ETA: 20:31 - loss: 0.0231 - acc: 0.99 - ETA: 20:31 - loss: 0.0231 - acc: 0.99 - ETA: 20:30 - loss: 0.0231 - acc: 0.99 - ETA: 20:30 - loss: 0.0232 - acc: 0.99 - ETA: 20:29 - loss: 0.0232 - acc: 0.99 - ETA: 20:29 - loss: 0.0232 - acc: 0.99 - ETA: 20:29 - loss: 0.0232 - acc: 0.99 - ETA: 20:29 - loss: 0.0231 - acc: 0.99 - ETA: 20:28 - loss: 0.0231 - acc: 0.99 - ETA: 20:28 - loss: 0.0231 - acc: 0.99 - ETA: 20:27 - loss: 0.0232 - acc: 0.99 - ETA: 20:27 - loss: 0.0232 - acc: 0.99 - ETA: 20:27 - loss: 0.0231 - acc: 0.99 - ETA: 20:27 - loss: 0.0231 - acc: 0.99 - ETA: 20:27 - loss: 0.0231 - acc: 0.99 - ETA: 20:26 - loss: 0.0232 - acc: 0.99 - ETA: 20:26 - loss: 0.0233 - acc: 0.99 - ETA: 20:26 - loss: 0.0232 - acc: 0.99 - ETA: 20:25 - loss: 0.0232 - acc: 0.99 - ETA: 20:25 - loss: 0.0232 - acc: 0.99 - ETA: 20:25 - loss: 0.0233 - acc: 0.99 - ETA: 20:25 - loss: 0.0233 - acc: 0.99 - ETA: 20:24 - loss: 0.0233 - acc: 0.

 612/3800 [===>..........................] - ETA: 19:23 - loss: 0.0234 - acc: 0.99 - ETA: 19:22 - loss: 0.0233 - acc: 0.99 - ETA: 19:22 - loss: 0.0234 - acc: 0.99 - ETA: 19:22 - loss: 0.0234 - acc: 0.99 - ETA: 19:22 - loss: 0.0234 - acc: 0.99 - ETA: 19:21 - loss: 0.0234 - acc: 0.99 - ETA: 19:21 - loss: 0.0234 - acc: 0.99 - ETA: 19:20 - loss: 0.0234 - acc: 0.99 - ETA: 19:20 - loss: 0.0235 - acc: 0.99 - ETA: 19:20 - loss: 0.0234 - acc: 0.99 - ETA: 19:19 - loss: 0.0234 - acc: 0.99 - ETA: 19:19 - loss: 0.0234 - acc: 0.99 - ETA: 19:19 - loss: 0.0234 - acc: 0.99 - ETA: 19:18 - loss: 0.0234 - acc: 0.99 - ETA: 19:18 - loss: 0.0234 - acc: 0.99 - ETA: 19:18 - loss: 0.0234 - acc: 0.99 - ETA: 19:17 - loss: 0.0234 - acc: 0.99 - ETA: 19:17 - loss: 0.0234 - acc: 0.99 - ETA: 19:17 - loss: 0.0234 - acc: 0.99 - ETA: 19:16 - loss: 0.0234 - acc: 0.99 - ETA: 19:16 - loss: 0.0234 - acc: 0.99 - ETA: 19:16 - loss: 0.0233 - acc: 0.99 - ETA: 19:15 - loss: 0.0233 - acc: 0.99 - ETA: 19:15 - loss: 0.0233 - acc: 0.

 816/3800 [=====>........................] - ETA: 18:14 - loss: 0.0232 - acc: 0.99 - ETA: 18:14 - loss: 0.0233 - acc: 0.99 - ETA: 18:14 - loss: 0.0232 - acc: 0.99 - ETA: 18:13 - loss: 0.0232 - acc: 0.99 - ETA: 18:13 - loss: 0.0232 - acc: 0.99 - ETA: 18:13 - loss: 0.0232 - acc: 0.99 - ETA: 18:12 - loss: 0.0232 - acc: 0.99 - ETA: 18:12 - loss: 0.0232 - acc: 0.99 - ETA: 18:12 - loss: 0.0232 - acc: 0.99 - ETA: 18:11 - loss: 0.0232 - acc: 0.99 - ETA: 18:11 - loss: 0.0232 - acc: 0.99 - ETA: 18:11 - loss: 0.0232 - acc: 0.99 - ETA: 18:10 - loss: 0.0232 - acc: 0.99 - ETA: 18:10 - loss: 0.0232 - acc: 0.99 - ETA: 18:10 - loss: 0.0232 - acc: 0.99 - ETA: 18:09 - loss: 0.0232 - acc: 0.99 - ETA: 18:09 - loss: 0.0232 - acc: 0.99 - ETA: 18:09 - loss: 0.0232 - acc: 0.99 - ETA: 18:08 - loss: 0.0232 - acc: 0.99 - ETA: 18:08 - loss: 0.0231 - acc: 0.99 - ETA: 18:08 - loss: 0.0231 - acc: 0.99 - ETA: 18:07 - loss: 0.0231 - acc: 0.99 - ETA: 18:07 - loss: 0.0231 - acc: 0.99 - ETA: 18:06 - loss: 0.0231 - acc: 0.

...

3060/3800 [=======================>......] - ETA: 5:24 - loss: 0.0228 - acc: 0.992 - ETA: 5:24 - loss: 0.0228 - acc: 0.992 - ETA: 5:23 - loss: 0.0228 - acc: 0.992 - ETA: 5:23 - loss: 0.0228 - acc: 0.992 - ETA: 5:23 - loss: 0.0228 - acc: 0.992 - ETA: 5:22 - loss: 0.0228 - acc: 0.992 - ETA: 5:22 - loss: 0.0228 - acc: 0.992 - ETA: 5:22 - loss: 0.0228 - acc: 0.992 - ETA: 5:21 - loss: 0.0228 - acc: 0.992 - ETA: 5:21 - loss: 0.0228 - acc: 0.992 - ETA: 5:21 - loss: 0.0228 - acc: 0.992 - ETA: 5:20 - loss: 0.0228 - acc: 0.992 - ETA: 5:20 - loss: 0.0228 - acc: 0.992 - ETA: 5:19 - loss: 0.0228 - acc: 0.992 - ETA: 5:19 - loss: 0.0228 - acc: 0.992 - ETA: 5:19 - loss: 0.0228 - acc: 0.992 - ETA: 5:18 - loss: 0.0228 - acc: 0.992 - ETA: 5:18 - loss: 0.0228 - acc: 0.992 - ETA: 5:18 - loss: 0.0228 - acc: 0.992 - ETA: 5:17 - loss: 0.0228 - acc: 0.992 - ETA: 5:17 - loss: 0.0228 - acc: 0.992 - ETA: 5:17 - loss: 0.0228 - acc: 0.992 - ETA: 5:16 - loss: 0.0228 - acc: 0.992 - ETA: 5:16 - loss: 0.0228 - acc: 0.9

3264/3800 [========================>.....] - ETA: 4:14 - loss: 0.0228 - acc: 0.992 - ETA: 4:13 - loss: 0.0228 - acc: 0.992 - ETA: 4:13 - loss: 0.0228 - acc: 0.992 - ETA: 4:13 - loss: 0.0228 - acc: 0.992 - ETA: 4:12 - loss: 0.0228 - acc: 0.992 - ETA: 4:12 - loss: 0.0228 - acc: 0.992 - ETA: 4:12 - loss: 0.0228 - acc: 0.992 - ETA: 4:11 - loss: 0.0228 - acc: 0.992 - ETA: 4:11 - loss: 0.0228 - acc: 0.992 - ETA: 4:11 - loss: 0.0228 - acc: 0.992 - ETA: 4:10 - loss: 0.0228 - acc: 0.992 - ETA: 4:10 - loss: 0.0228 - acc: 0.992 - ETA: 4:10 - loss: 0.0228 - acc: 0.992 - ETA: 4:09 - loss: 0.0228 - acc: 0.992 - ETA: 4:09 - loss: 0.0228 - acc: 0.992 - ETA: 4:09 - loss: 0.0228 - acc: 0.992 - ETA: 4:08 - loss: 0.0228 - acc: 0.992 - ETA: 4:08 - loss: 0.0228 - acc: 0.992 - ETA: 4:08 - loss: 0.0228 - acc: 0.992 - ETA: 4:07 - loss: 0.0228 - acc: 0.992 - ETA: 4:07 - loss: 0.0228 - acc: 0.992 - ETA: 4:07 - loss: 0.0228 - acc: 0.992 - ETA: 4:06 - loss: 0.0228 - acc: 0.992 - ETA: 4:06 - loss: 0.0228 - acc: 0.9

3468/3800 [==========================>...] - ETA: 3:04 - loss: 0.0228 - acc: 0.992 - ETA: 3:03 - loss: 0.0228 - acc: 0.992 - ETA: 3:03 - loss: 0.0228 - acc: 0.992 - ETA: 3:03 - loss: 0.0228 - acc: 0.992 - ETA: 3:02 - loss: 0.0228 - acc: 0.992 - ETA: 3:02 - loss: 0.0228 - acc: 0.992 - ETA: 3:02 - loss: 0.0228 - acc: 0.992 - ETA: 3:01 - loss: 0.0228 - acc: 0.992 - ETA: 3:01 - loss: 0.0228 - acc: 0.992 - ETA: 3:01 - loss: 0.0228 - acc: 0.992 - ETA: 3:00 - loss: 0.0229 - acc: 0.992 - ETA: 3:00 - loss: 0.0229 - acc: 0.992 - ETA: 3:00 - loss: 0.0228 - acc: 0.992 - ETA: 2:59 - loss: 0.0228 - acc: 0.992 - ETA: 2:59 - loss: 0.0228 - acc: 0.992 - ETA: 2:58 - loss: 0.0228 - acc: 0.992 - ETA: 2:58 - loss: 0.0228 - acc: 0.992 - ETA: 2:58 - loss: 0.0228 - acc: 0.992 - ETA: 2:57 - loss: 0.0228 - acc: 0.992 - ETA: 2:57 - loss: 0.0228 - acc: 0.992 - ETA: 2:57 - loss: 0.0228 - acc: 0.992 - ETA: 2:56 - loss: 0.0228 - acc: 0.992 - ETA: 2:56 - loss: 0.0229 - acc: 0.992 - ETA: 2:56 - loss: 0.0229 - acc: 0.9

3675/3800 [============================>.] - ETA: 1:53 - loss: 0.0228 - acc: 0.992 - ETA: 1:53 - loss: 0.0228 - acc: 0.992 - ETA: 1:53 - loss: 0.0228 - acc: 0.992 - ETA: 1:52 - loss: 0.0228 - acc: 0.992 - ETA: 1:52 - loss: 0.0228 - acc: 0.992 - ETA: 1:52 - loss: 0.0228 - acc: 0.992 - ETA: 1:51 - loss: 0.0228 - acc: 0.992 - ETA: 1:51 - loss: 0.0228 - acc: 0.992 - ETA: 1:51 - loss: 0.0228 - acc: 0.992 - ETA: 1:50 - loss: 0.0228 - acc: 0.992 - ETA: 1:50 - loss: 0.0228 - acc: 0.992 - ETA: 1:50 - loss: 0.0228 - acc: 0.992 - ETA: 1:49 - loss: 0.0228 - acc: 0.992 - ETA: 1:49 - loss: 0.0228 - acc: 0.992 - ETA: 1:49 - loss: 0.0228 - acc: 0.992 - ETA: 1:48 - loss: 0.0228 - acc: 0.992 - ETA: 1:48 - loss: 0.0228 - acc: 0.992 - ETA: 1:48 - loss: 0.0228 - acc: 0.992 - ETA: 1:47 - loss: 0.0228 - acc: 0.992 - ETA: 1:47 - loss: 0.0228 - acc: 0.992 - ETA: 1:47 - loss: 0.0228 - acc: 0.992 - ETA: 1:46 - loss: 0.0228 - acc: 0.992 - ETA: 1:46 - loss: 0.0228 - acc: 0.992 - ETA: 1:46 - loss: 0.0228 - acc: 0.9

3800/3800 [==============================] - ETA: 42s - loss: 0.0228 - acc: 0.99 - ETA: 42s - loss: 0.0228 - acc: 0.99 - ETA: 42s - loss: 0.0228 - acc: 0.99 - ETA: 41s - loss: 0.0228 - acc: 0.99 - ETA: 41s - loss: 0.0228 - acc: 0.99 - ETA: 40s - loss: 0.0228 - acc: 0.99 - ETA: 40s - loss: 0.0228 - acc: 0.99 - ETA: 40s - loss: 0.0228 - acc: 0.99 - ETA: 39s - loss: 0.0228 - acc: 0.99 - ETA: 39s - loss: 0.0228 - acc: 0.99 - ETA: 39s - loss: 0.0228 - acc: 0.99 - ETA: 38s - loss: 0.0228 - acc: 0.99 - ETA: 38s - loss: 0.0228 - acc: 0.99 - ETA: 38s - loss: 0.0228 - acc: 0.99 - ETA: 37s - loss: 0.0228 - acc: 0.99 - ETA: 37s - loss: 0.0228 - acc: 0.99 - ETA: 37s - loss: 0.0228 - acc: 0.99 - ETA: 36s - loss: 0.0228 - acc: 0.99 - ETA: 36s - loss: 0.0228 - acc: 0.99 - ETA: 36s - loss: 0.0228 - acc: 0.99 - ETA: 35s - loss: 0.0228 - acc: 0.99 - ETA: 35s - loss: 0.0228 - acc: 0.99 - ETA: 35s - loss: 0.0228 - acc: 0.99 - ETA: 34s - loss: 0.0228 - acc: 0.99 - ETA: 34s - loss: 0.0228 - acc: 0.99 - ETA: 

In [36]:
for layer in model3.layers:
  print (colored(layer.name, 'blue'),colored(layer.trainable, 'red'))

conv2d_7 False
activation_9 False
conv2d_8 False
activation_10 False
conv2d_9 False
activation_11 False
max_pooling2d_3 False
dropout_6 False
batch_normalization_6 False
conv2d_10 False
activation_12 False
conv2d_11 False
activation_13 False
dropout_7 False
batch_normalization_7 False
max_pooling2d_4 False
dropout_8 False
batch_normalization_8 False
conv2d_12 False
activation_14 False
dropout_9 False
batch_normalization_9 False
flatten_2 False
dense_3 True
activation_15 False
dropout_10 False
batch_normalization_10 False
dense_4 True
activation_16 False


In [37]:
model3.evaluate(x_train,y_train)

3800/3800 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 288us/step


[0.028606982581205604, 0.9918421052631579]

In [38]:
model3.evaluate(x_test,y_test)

950/950 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 306us/step


[0.3014300198284419, 0.9473684216800489]

## After Transfer learning there seems to be an increase in appox 1% in both train and test evalatuation. Final accuracy in train is 99.18 and 94.74 in test